In [1]:
# Libraries
import itertools
from importlib import resources
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
import time

from drdt.helper_functions import DecisionRuleCreatorFromDecisionTable, Reduction, R_SR, R_AD, SAlphaStep, SPlus, SMax, NCover, NGreedy
from drdt.algorithms import DynamicProgrammingAlgorithms, A_C_N, A_C_G


# Loading Data

In [2]:
with resources.path('datasets.DecisionRuleSystems', 'DRS_breast_cancer') as dataset_path:
    S = pd.read_csv(dataset_path).applymap(lambda x: str(x) if pd.notnull(x) else x)
S = S.dropna(axis=1, how='all') # Drop the columns with all None
S

/var/folders/06/0m2hb6f927s6rdyhxjrbss740r2h_1/T/ipykernel_85097/4260445852.py:1: DeprecationWarning: path is deprecated. Use files() instead. Refer to https://importlib-resources.readthedocs.io/en/latest/using.html#migrating-from-legacy for migration advice.
  with resources.path('datasets.DecisionRuleSystems', 'DRS_breast_cancer') as dataset_path:
/var/folders/06/0m2hb6f927s6rdyhxjrbss740r2h_1/T/ipykernel_85097/4260445852.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  S = pd.read_csv(dataset_path).applymap(lambda x: str(x) if pd.notnull(x) else x)


,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,class
0,20-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no-recurrence-events
1,30-39,lt40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no-recurrence-events
2,NaN,NaN,0-4,NaN,NaN,NaN,NaN,NaN,NaN,no-recurrence-events
3,30-39,NaN,14-Oct,NaN,NaN,NaN,NaN,NaN,NaN,no-recurrence-events
4,30-39,NaN,14-Oct,NaN,NaN,NaN,NaN,NaN,NaN,no-recurrence-events
...,...,...,...,...,...,...,...,...,...,...
261,70-79,NaN,14-Oct,NaN,NaN,NaN,NaN,NaN,NaN,no-recurrence-events
262,70-79,NaN,NaN,11-Sep,NaN,NaN,NaN,NaN,NaN,recurrence-events
263,70-79,NaN,20-24,NaN,NaN,NaN,NaN,NaN,NaN,no-recurrence-events
264,70-79,NaN,40-44,NaN,NaN,NaN,NaN,NaN,NaN,no-recurrence-events


# Dataset Analyses

In [3]:
n = len(S.columns)-1
print(f"Number of features  = {n}")

Number of features  = 9


In [4]:
print(f"Does # of features = d  = {not S.dropna().empty}")

Does # of features = d  = False


In [5]:
# Count non-NaN values for each row
non_nan_counts = S.count(axis=1)

# Find the index
max_non_nan_row_index = non_nan_counts.idxmax()

# Retrieve the row
max_non_nan_row = S.loc[max_non_nan_row_index]

# Number of non-NaN values in the row
max_non_nan_count = non_nan_counts[max_non_nan_row_index]

d = max_non_nan_count - 1

print(f"d = {d}") # remove 1 because of last class column

d = 6


In [6]:
P = S
P_plus = SPlus(P)
B = NCover(P_plus)
print(f"Length of Node Cover for AR = {len(B)}")

Length of Node Cover for AR = 6


In [7]:
P = R_SR(S)
P_plus = SPlus(P)
B = NCover(P_plus)
print(f"Length of Node Cover for SR = {len(B)}")

Length of Node Cover for SR = 6


In [8]:
P = R_AD(S)
P_plus = SPlus(P)
B = NCover(P_plus)
print(f"Length of Node Cover for AD = {len(B)}")

Length of Node Cover for AD = 6


In [9]:
P = S
P_plus = SPlus(P)
B = NGreedy(P_plus)
print(f"Length of Node Cover greedy for AR = {len(B)}")

Length of Node Cover greedy for AR = 1


In [10]:
P = R_SR(S)
P_plus = SPlus(P)
B = NGreedy(P_plus)
print(f"Length of Node Cover greedy for SR = {len(B)}")

Length of Node Cover greedy for SR = 1


In [11]:
P = R_AD(S)
P_plus = SPlus(P)
B = NGreedy(P_plus)
print(f"Length of Node Cover greedy for AD = {len(B)}")

Length of Node Cover greedy for AD = 1


# Combinations

In [12]:
column_values = []

for column in S.columns[:-1]: #ignoring class column
    column_values.append(list(S[column].dropna().unique()))
    
# All possible combinations    
combinations = list(itertools.product(*column_values))

print('Number of Possible combinations =', len(combinations))

Number of Possible combinations = 166320


In [13]:
# # calculate 10% sample size
# sample_size = int(len(combinations)*0.1)

# # Get a random sapmple of combinations
# combinations = random.sample(combinations, sample_size)

# print('Number of Possible combinations after sampling =', len(combinations))

# Node Cover Greedy

In [14]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AD", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("NCgreedy AD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 166320/166320 [1:09:28<00:00, 39.90it/s]

NCgreedy AD
(Max Depth, Min Depth, Average Depth, Running time) = (9, 3, 5.602417027417028, 4168.7433540821075)
